# Исследование объявлений о продаже квартир

## Описание задачи

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и Ленинградской области за несколько лет. 

Данные содержат различные параметры объявлений о продаже недвижимости: цена предложения, габариты квартиры/отдельных помещений, характеристики дома, этаж и этажность, расположение и расстояние до ключевых инфраструктурных объектов (крупного города, парка, магазина и т.д.)

## Детальное описание данных

* airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
* balcony — число балконов
* ceiling_height — высота потолков (м)
* cityCenters_nearest — расстояние до центра города (м)
* days_exposition — сколько дней было размещено объявление (от публикации до снятия)
* first_day_exposition — дата публикации
* floor — этаж
* floors_total — всего этажей в доме
* is_apartment — апартаменты (булев тип)
* kitchen_area — площадь кухни в квадратных метрах (м²)
* last_price — цена на момент снятия с публикации
* living_area — жилая площадь в квадратных метрах (м²)
* locality_name — название населённого пункта
* open_plan — свободная планировка (булев тип)
* parks_around3000 — число парков в радиусе 3 км
* parks_nearest — расстояние до ближайшего парка (м)
* ponds_around3000 — число водоёмов в радиусе 3 км
* ponds_nearest — расстояние до ближайшего водоёма (м)
* rooms — число комнат
* studio — квартира-студия (булев тип)
* total_area — общая площадь квартиры в квадратных метрах (м²)
* total_images — число фотографий квартиры в объявлении

## Что мы можем сделать с этими данными и чем они полезны бизнесу?

* Данные по объявлениям помогают нам принимать решения о покупке квартиры в различных целях: для проживания или инвестиции
* Данные по объявлениям помогают бизнесу оценивать проекты в сфере недвижимости
* Сервисам по объялениям данные помогают оценивать рыночное состояние рынка недвижимости и предотвращать мошенничество
* Для дата-саентистов данные помогают строить передективные модели и понимать тенденции развития рынка

## Постановка задачи

* Провести обзор рынка недвижимости и первичную обработку данных
* Найти среднюю стоиомость квадратного метра жилья (любой другой расчетный параметр)
* Установить зависимость между ценой квадратного метра и расстоянием до центра (любые ваши параметры)
* Построить графики, отражающие данные о рынке недвижимости (любые ваши идеи и предложения)
* Принять решение: стоит ли приобретать/продавать объект недвижимости

## Шаги по достижению цели

* Откройте файл с данными и изучите общую информацию
* Выполните предобработку данных
* Проведите исследовательский анализ данных
* Напишите общий вывод по исследованию

# Ход решения задачи

## Шаг 0. Подготовка к работе

Выгрузим библиотеки, которые нам понадобятся в работе

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from pandasql import sqldf
import math

## Шаг 1. Получение данных из источника

Прочитаем файл:

In [ ]:
df = pd.read_csv('real_estate_data.csv', sep='\t')

Посмотрим, как у нас получилось загрузить данные:

In [ ]:
df.head()

Оставим только те признаки, которые хотим проанализировать:

In [ ]:
df = df[['last_price','total_area','first_day_exposition','rooms','ceiling_height','floors_total',\
          'floor','kitchen_area','balcony','locality_name','days_exposition','cityCenters_nearest']]
df.head()

Выведем информацию о фрейме данных:

In [ ]:
df.info()

Посмотрим все колонки этой таблицы:

In [ ]:
df.columns

Проверим, сколько строк по каждой колонке заполнены пустыми значениями:

In [ ]:
df.isna().sum()

### ВЫВОДЫ

* В данных 23699 записи. 
* Для целого ряда столбцов, есть пропущенные значения и некорректный тип данных.
* Данные признаны полными и достаточными для исследования.

## Шаг 2. Предобработка данных

#### Что мы можем сделать на этом этапе?

* Проверить дублирующие значения
* Повысить заполняемость признаков объектов недвижимости
* Изменить типы данных на требуемые нам
* Переименовать столбцы в удобные и короткие названия
* Убрать "битые" данные

Помните, что данные - это данность и мы их не можем "выдумать". Это критичное замечание влияет на все результаты исследований

Переименуем колонку ближайшего города:

In [ ]:
df.rename(columns = {'cityCenters_nearest':'city_centers_nearest'}, inplace = True )
df.head(5)

Переведем тип данных по количеству картинок в объявлении к типу число:

In [ ]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
df.head()

Аналогично, тип данных у признака "число этажаей в здании" также должен быть целым числом, а не вещественным как это есть сейчас. Исправим эту неточность:

In [ ]:
df['floors_total'] = df['floors_total'].fillna(df['floor']).astype(int)
df.head()

Переведем расстояние до ближайшего города из метров в километры и сохраним в той же колонке:

In [ ]:
df['city_centers_nearest'] = df['city_centers_nearest']/1000
df['city_centers_nearest'].round(2)
df.head()

Округлим общую площадь объектов до 2 знаков после запятой:

In [ ]:
df['total_area'].round(2)
df.head()

Заменим пустые значения высоты потолоков усредненным значением по всем объектам:

In [ ]:
df['ceiling_height'] = df['ceiling_height'].fillna(df['ceiling_height'].median())
df.head()

Заменим этажность домов усредненным значением по всем объектам:

In [ ]:
def floors_total(row):
    if row['floors_total'] >= 0:
        return 0
    else:
        return df[df['floor'] == row['floor']]['floors_total'].median()
        
df['f'] = df.apply(floors_total, axis=1) 
df['floors_total'] = df['floors_total'].fillna(0)
df['floors_total'] = (df['f'] + df['floors_total']).astype('int')
df.drop('f', axis= 1 , inplace= True )
df['floors_total'].unique()

Посчитаем дубли в датасете:

In [ ]:
df.duplicated().sum()

Если бы в датасете были дубли, мы бы их могли убрать простой командой:

In [ ]:
df.drop_duplicates()
df.head()

Заполним пустоты в данных о наличии балконов нулевыми значениями:

In [ ]:
df['balcony'] = df['balcony'].fillna(0)

Посчитаем, какая доля объявлений имеет незаполненную локацию. Доля таких строчек стала нулевой.

In [ ]:
(df['locality_name'].isna().sum() / df.shape[0]).round(4)

Заполним пропущенные названия населенных пунктов:

In [ ]:
df['locality_name'] = df['locality_name'].fillna('unknown')

Мы могли бы также удалить объявления с пустыми локациями объектов:

In [ ]:
df = df.dropna(subset=['locality_name'])
df.head()

В качестве эксперимента мы можем придумать и более сложную логику для доопределения недостающих данных. Например, посчитать среднюю высоту объектов в домах различной высотности и заполнить пустые параметры высоты потолков:

In [ ]:
ceiling_height_median = df.groupby('floor')['ceiling_height'].transform('median')
df['ceiling_height'] = df['ceiling_height'].fillna(ceiling_height_median)
df['ceiling_height'] = df['ceiling_height'].fillna(df['ceiling_height'].mode())

df.head()

Или вот еще вариант. Можем рассчитать недостающий данные по площади кухонь,у среднив значения по аналогам в разрезе общей площади и количества комнат:

In [ ]:
kitchen_height_median = df.groupby(['total_area','rooms'])['kitchen_area'].transform('median')
df['kitchen_area'] = df['kitchen_area'].fillna(kitchen_height_median)
df['kitchen_area'] = df['kitchen_area'].fillna(df['kitchen_area'].mode())

df.head()

## ВЫВОД:
* Убрали дупликаты из таблицы (сначала проверили, что их нет, и на всякий случай удалили)
* Сделали преобразования типов данных (перевели метры в километры, превратили вещественные числа в целые)
* Улучшили заполнение некоторых признаков (заполнили пустоты по числу балконов, усреднили площади кухонь и высотности квартир)
* Убрали битые данные (строки, где не была заполнена локация)

## Шаг 3. Начнем анализировать данные по недвижимости

### Что происхдит на этом этапе?

* Вычисляем необходимые параметры
* Строим репрезентативные графики
* Выявляем тендеции и закономерности
* Проводим анализ зависимости параметров друг от друга 

Опционально:
* Смторим на данные с различных разрезах
* Ставим эксперименты и выдивигаем гипотезы

Добавим новые вспомогательные полонки в датасет, отражающие год, месяц и неделю, основываясь на дате размещения объявления:

In [ ]:
df['week_day'] = pd.DatetimeIndex(df['first_day_exposition']).weekday
df['year'] = pd.DatetimeIndex(df['first_day_exposition']).year
df['month'] = pd.DatetimeIndex(df['first_day_exposition']).month

df.info()

Посмотрим уникальные даты размещения объявлений:

In [ ]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
df['first_day_exposition'].unique()

### Анализ цены за квадратный метр жилья

Добавим столбец price_square_meter с ценой одного квадратного метра объекта с точностью до двух знаков после запятой:

In [ ]:
df['price_on_metr'] = (df['last_price'] / df['total_area']).round(2).astype(int)
df.head()

Также можем посмотреть цены на квадратный метр отдельно по Санкт-Петербургу и Ленинградской области:

In [ ]:
df_piter = df.query('locality_name == "Санкт-Петербург"')
df_len_obl = df.query('locality_name != "Санкт-Петербург"')

In [ ]:
df_piter['price_on_metr'].hist(figsize=(10,4), bins=40, alpha=0.5)
df_len_obl['price_on_metr'].hist(figsize=(10,4), bins=40, alpha=0.5)
plt.legend(['Питер', 'Лен область'])
plt.title('Гистограммы Питер и область цена за 1 кв.м.')
plt.show()

In [ ]:
df_piter['price_on_metr'].mean()

In [ ]:
df_len_obl['price_on_metr'].mean()

#### Вывод:
Цены в Санкт-Петербурге ожидаемо выше, чем в Ленинградской области

### Анализ общей площади объектов

In [ ]:
df['total_area'].describe()

Оставим значения от минимального 12 до 100 кв. метров построим гистограмму

In [ ]:
plt.hist(df['total_area'], bins=100, range=(12,100))
plt.title('Зависимость квартир в продаже от площади квартир')
plt.xlabel('Квадратные метры')
plt.ylabel('Количество квартир')
plt.show()

#### ВЫВОД: 
На рынке самое большое предложение квартир с общей площадью от 40 до 69 кв. метров с максимумом около 52 кв. метров.

### Анализ локаций

In [ ]:
df['locality_name'].value_counts().head(5).plot(kind = 'barh',figsize=(8,4))
plt.title('Распределение по городам ТОП 5')
plt.show()

#### Вывод:
Больше всего объектов размещено ожидаемо в Санкт-Петербурге

### Анализ площади кухни

In [ ]:
df['kitchen_area'].describe()

In [ ]:
plt.hist(df['kitchen_area'], bins=100, range=(6,12))
plt.title('Зависимость квартир в продаже от площади кухни')
plt.xlabel('Квадратные метры')
plt.ylabel('Количество квартир')
plt.show()

#### Вывод

Прослеживается тенденция что больше всего предложений с площадью кухни в 6 метров затем 10 и чередом 7,8,9 кв. метров, при этом в анкетах заметны явные всплески на целых числах.

### Анализ этажности

Ни для кого не секрет, что объекты часто рассматриваются по 3 категориям в засимости от этажа: первый, последний, все остальные.
Давайте так же классифицируем наши объекты

In [ ]:
df['floor_category'] = np.where(df['floor'] == 1, 'Первый',
                       np.where(df['floor'] == df['floors_total'],'Посдедний','Другой'))
df.groupby('floor_category').size().plot(kind = 'pie', autopct='%1.0f%%')
plt.title('Круговая диаграмма категории этажа')
plt.show()

#### Вывод
Менее ценными считаются квартиры на "Первом" и "Последнем" этаже. На круговой диаграмме заметно, что общее количество предложений "Первого" и "Последнего" этажа составляет чуть больше 25% от общего количества предложений, значит люди хотят сьехать с этих этажей.

### Анализ цены

In [ ]:
df['last_price'].describe()

In [ ]:
plt.hist(df['last_price'], bins=100, range=(1.219000e+04,9.00000e+06))
plt.title('Зависимость квартир в продаже от цены на момент снятия с публикации')
plt.xlabel('Цена в млн. руб.')
plt.ylabel('Количество квартир')
plt.show()

#### Вывод:
Из гисторгаммы следует, что наиболее часто цена объекта составляет 3,8 - 4,2 миллниона.

### Анализ комнатности

In [ ]:
plt.hist(df['rooms'], bins=7, range=(0,7))
plt.title('Зависимость квартир в продаже от комнат')
plt.xlabel('Количество комнат')
plt.ylabel('Количество квартир')
plt.show()

#### Вывод:
Среди предложений больше всего однокомнатных квартир. С увеличением комнат количество предложений снижается почти линейно.

### Анализ высоты потолков

In [ ]:
df['ceiling_height'].describe()

In [ ]:
plt.hist(df.query('(ceiling_height > 2) & (ceiling_height < 4)')['ceiling_height'], bins=50, range=(2.4,3))
plt.title('Зависимость квартир в продаже от высоты потолков')
plt.xlabel('Метры')
plt.ylabel('Количество квартир')
plt.show()

#### Вывод
Среди большинства предложений высота составляет 2.65 метров, далее 2.5, 2.6, 2.7, 2.55 метров.

### Анализ размещений объявлений

Построим график распределения частоты объявлений по годам, месяцам и нееделям

In [ ]:
def hist_df(data):
    for column,name in zip(['year','week_day','month'],['Годы','День недели','Месяц']):
        data.plot(y = column, kind = 'hist', bins = 40, figsize = (8,4))
        plt.title(name)
    return plt.show()

In [ ]:
for column,name in zip(['year','week_day','month'],['Годы','День недели','Месяц']):
    df.plot(y = column, kind = 'hist', bins = 40, figsize = (8,4))
    plt.title(name)
    plt.show()

### Вывод

* Годы отличаются явно сезонностью в 2017 и 2018 гг. Возможно, что данные за 2019 год неполный, а ранее сервис Я.Недвижимость еще не существовал
* В днях недели также выявлена явная сезонность: в выходные дни реже происходят размещения и продажи недвижимости
* В месяцах также заметна сезонность: в январе и мае (традиционно праздничные месяцы в России) сделок и объявлений меньше, чем в другие сезоны. Пик сделок приходится на конец зимы - начало весны и осень. Лето также традиционно "не сезон" на рынке недвижимости.

### Корреляционный анализ

Давайте посмотрим на то, как связаны между собой различные ключевые показатели квартир

In [ ]:
df_piter['city_centers_nearest'].corr(df_piter['price_on_metr'])

In [ ]:
plt.figure(figsize=(5,5))
sns.heatmap(df_piter[['price_on_metr','total_area','rooms','ceiling_height', 'city_centers_nearest']]\
               .corr(), vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', linewidths=3, linecolor='black')
plt.show()

#### Вывод
Существует сильная прямая связь между числом комнат и общей площадью квартир
Существует сильная обратная связь между ценой за квадратный метр и удаленностью от центра города

### Анализ зависимости цены квадратного метра от удаленности от города

In [ ]:
df_piter['km_center'] = (df_piter['city_centers_nearest']).round()
df_piter['km_center'].hist(bins = 25)

In [ ]:
df_piter.groupby('km_center')['price_on_metr'].mean().plot(figsize=(10,4), style='o-', grid = True)
plt.title('Зависимость цены от удаленности от центра')
plt.show()

#### Вывод:
Для поиска оптимального соотношения "цена-расстояние" следует обратить внимание на объекты в пределах 2-4 или 7-10 км от центра города

### Выводы по всему разделу:
* На рынке самое большое предложение квартир с общей площадью от 40 до 69 кв. метров с максимумом около 52 кв. метров.
* Больше всего продают квартир с жилой площадью около 18 кв. м и 30 кв. метров при этом прослеживается спад предложений в средней зоне 23 кв. метра.
* Прослеживается тенденция что больше всего предложений с площадью кухни в 6 метров затем 10 и чередом 7,8,9 кв. метров, при этом в анкетах заметны явные всплески на целых числах.
* Из гисторгаммы следует, что наиболее часто цена объекта составляет 3,8 - 4,2 миллниона.
* Среди предложений больше всего однокомнатных квартир. С увеличением комнат количество предложений снижается почти линейно.
* Среди большинства предложений высота составляет 2.65 метров, далее 2.5, 2.6, 2.7, 2.55 метров.
* Менее ценными считаются квартиры на "первом" этаже. На круговой диаграмме заметно, что общее количество предложений "первого" и "последнего" этажа составляет почти 25% от общего количества, значит люди хотят сьехать с этих этажей.
* Больше всего предложений о продаже квартир в пятиэтажных домах, а потом в девятиэтажках.
* Больше всего предложений о продаже в спальных районах на окраинах города (на расстоянии 12000 -15000 метров), и значительным всплеском на расстоянии в 5000 метров от центра города.

### Опциональные возможности

In [ ]:
q = """ select * from df_piter """
sqldf(q)